In [37]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224, usb=0)

image = widgets.Image(format='jpeg', width=400, height=400)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
classfication_result_image = widgets.Textarea(
            value=u'Result:=',
            description=u'Classifcation',
            disabled=False)
display(widgets.VBox([
    widgets.HBox([image, classfication_result_image]),
]))


In [3]:
def rgba2rgb(rgba, background=(255,255,255)):
    row, col, ch = rgba.shape
    if ch == 3:
        return rgba
    assert ch == 4, "RGBA image has 4 channels"
    rgb = np.zeros((row, col, 3), dtype='float32')
    r, g, b, a = rgba[:,:,0], rgba[:,:,1], rgba[:,:,2], rgba[:,:,3]
    a = np.asarray(a, dtype='float32') / 255.0
    R, G, B = background
    rgb[:,:,0] = r * a + (1.0 - a) * R
    rgb[:,:,1] = g * a + (1.0 - a) * G
    rgb[:,:,2] = b * a + (1.0 - a) * B
    return np.asarray(rgb, dtype="uint8")

In [2]:
import tensorflow as tf

# Load savedmodel from the Google's Teachable Machine Web site
saved_model = tf.keras.models.load_model("savedmodel")
nano_model = saved_model.signatures["serving_default"]

In [27]:
nano_image=rgba2rgb(camera.value)

In [29]:
prediction = nano_model(tf.constant(nano_image, dtype=float))["sequential_3"].numpy()


ValueError: The argument 'sequential_1_input' (value Tensor("Const_1:0", shape=(480, 640, 3), dtype=float32)) is not compatible with the shape this function was traced with. Expected shape (?, 224, 224, 3), but got shape (480, 640, 3).

If you called get_concrete_function, you may need to pass a tf.TensorSpec(..., shape=...) with a less specific shape, having None on axes which can vary.

In [25]:
from datetime import datetime

def execute(change):
    image = change['new']
#    output = model_trt(preprocess(camera.value).half())[0].detach().cpu().float().numpy()
#    mask=decode_segmap(output.argmax(0))
    #mask = 1.0 * (output.argmax(0) == 15)
    #m = mask[:, :, None]
    #print(m.shape)
    #print(image.shape)
    #seg_image.value = bgr8_to_jpeg(mask[:, :, None] * image)
    today = datetime.now()

    classfication_result_image.value = "Text" + today.strftime("%d/%m/%Y %H:%M:%S")
    
mask = execute({'new': camera.value})
camera.observe(execute, names='value')

In [26]:
camera.stop()

In [38]:
from tensorflow.keras.models import load_model

from PIL import Image, ImageOps
import numpy as np

In [39]:

model = load_model('keras_model.h5')

ValueError: Unknown layer: Functional